## Capstone 1
To import the dataset I have simply copied the table from wikipedia into excel and save it as a csv file

In [1]:
#!conda install -c conda-forge geopy --yes 
#!conda install -c conda-forge folium=0.5.0 --yes

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
#%matplotlib notebook
%matplotlib inline
import folium
from geopy.geocoders import Nominatim
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Libraries Imported")

Libraries Imported


## Read and check the dataset

In [3]:
df1=pd.read_csv("C:\\Users\\Piero\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Input_Table_01.csv")
print(df1.head())
print(df1.shape)
print(df1.columns)

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
(288, 3)
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


### Eliminating the rows that don't have an assigned Borough and assigning the name of the Borough to those remaining rows with a not assigned Neighbourhood

In [4]:
print('Before Splitting',np.unique(df1['Borough'].values))
df2=df1[df1['Borough']!='Not assigned']
print('Before Splitting',np.unique(df2['Borough'].values))
print('size of df2: ',df2.shape)
positions=[]
for i in range(len(df2)):
    if df2['Neighbourhood'].iloc[i]=='Not assigned':
        positions.append(i)
print('Before Modification',df2.iloc[positions])
for i in range(len(df2)):
    if df2['Neighbourhood'].iloc[i]=='Not assigned':
        df2['Neighbourhood'].iloc[i]= df2['Borough'].iloc[i]
print('After Modification',df2.iloc[positions])

Before Splitting ['Central Toronto' 'Downtown Toronto' 'East Toronto' 'East York'
 'Etobicoke' 'Mississauga' 'North York' 'Not assigned' "Queen's Park"
 'Scarborough' 'West Toronto' 'York']
Before Splitting ['Central Toronto' 'Downtown Toronto' 'East Toronto' 'East York'
 'Etobicoke' 'Mississauga' 'North York' "Queen's Park" 'Scarborough'
 'West Toronto' 'York']
size of df2:  (211, 3)
Before Modification   Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned
After Modification   Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Queen's Park


C:\Users\Piero\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [5]:
gropus_by_Postcode=df2.groupby(by='Postcode')
postcode=[]
borough=[]
neig=[]
for name,value in gropus_by_Postcode:
    postcode.append(value['Postcode'].iloc[0])
    borough.append(value['Borough'].iloc[0])
    neig.append(', '.join(value['Neighbourhood']))
df3=pd.DataFrame({'Postcode':postcode,'Borough':borough,'Neighbourhood':neig})
print(df3.head())
print(df3.shape)
print(df3.columns)

  Postcode      Borough                           Neighbourhood
0      M1B  Scarborough                          Rouge, Malvern
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2      M1E  Scarborough       Guildwood, Morningside, West Hill
3      M1G  Scarborough                                  Woburn
4      M1H  Scarborough                               Cedarbrae
(103, 3)
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')


### Checking that the raws showed on the excerices explanation have the same values of my dataset

In [6]:
df3[df3['Postcode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
5,M1J,Scarborough,Scarborough Village
11,M1R,Scarborough,"Maryvale, Wexford"
17,M2H,North York,Hillcrest Village
35,M4B,East York,"Woodbine Gardens, Parkview Hill"
38,M4G,East York,Leaside
43,M4M,East Toronto,Studio District
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"
57,M5G,Downtown Toronto,Central Bay Street
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [7]:
df3.shape

(103, 3)

## Loading the dataset with the geographical coordinates

In [8]:
df_coord=pd.read_csv("C:\\Users\\Piero\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Geospatial_Coordinates.csv")
df_coord.columns=['Postcode', 'Latitude', 'Longitude']
print(df_coord.head())
print(df_coord.shape)
print(df_coord.columns)

  Postcode   Latitude  Longitude
0      M1B  43.806686 -79.194353
1      M1C  43.784535 -79.160497
2      M1E  43.763573 -79.188711
3      M1G  43.770992 -79.216917
4      M1H  43.773136 -79.239476
(103, 3)
Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')


In [9]:
df_toronto=pd.merge(df3,df_coord,on='Postcode')
print(df_toronto.head())
print(df_toronto.shape)
print(df_toronto.columns)

  Postcode      Borough                           Neighbourhood   Latitude  \
0      M1B  Scarborough                          Rouge, Malvern  43.806686   
1      M1C  Scarborough  Highland Creek, Rouge Hill, Port Union  43.784535   
2      M1E  Scarborough       Guildwood, Morningside, West Hill  43.763573   
3      M1G  Scarborough                                  Woburn  43.770992   
4      M1H  Scarborough                               Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  
(103, 5)
Index(['Postcode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude'], dtype='object')


In [10]:
df_toronto[df_toronto['Postcode'].isin(['M5G','M2H','M4B','M1J','M4G','M4M','M1R','M9V','M9L','M5V','M1B','M5A'])]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
11,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
35,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
38,M4G,East York,Leaside,43.709060,-79.363452
43,M4M,East Toronto,Studio District,43.659526,-79.340923
53,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
68,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [11]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [12]:
# create map of New York using latitude and longitude values

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Selecting only the boroughs that contain the word Toronto.

In [13]:
df_tor_cut=df_toronto[df_toronto['Borough'].str.contains('Toronto')]
print(df_tor_cut.shape)
print(df_tor_cut.head())
print(df_tor_cut['Borough'].value_counts())

(38, 5)
   Postcode          Borough                   Neighbourhood   Latitude  \
37      M4E     East Toronto                     The Beaches  43.676357   
41      M4K     East Toronto    The Danforth West, Riverdale  43.679557   
42      M4L     East Toronto  The Beaches West, India Bazaar  43.668999   
43      M4M     East Toronto                 Studio District  43.659526   
44      M4N  Central Toronto                   Lawrence Park  43.728020   

    Longitude  
37 -79.293031  
41 -79.352188  
42 -79.315572  
43 -79.340923  
44 -79.388790  
Downtown Toronto    18
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64


## Starting Analysis

In [14]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '' # Foursquare API version


## Defining the function to analyse different neighborhoods.

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
tor_venues = getNearbyVenues(names=df_tor_cut['Neighbourhood'], latitudes=df_tor_cut['Latitude'], longitudes=df_tor_cut['Longitude'])


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [17]:
print(tor_venues.shape)
print(tor_venues.head())
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

(1705, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0  The Beaches              43.676357              -79.293031   
1  The Beaches              43.676357              -79.293031   
2  The Beaches              43.676357              -79.293031   
3  The Beaches              43.676357              -79.293031   
4  The Beaches              43.676357              -79.293031   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                 Glen Stewart Ravine       43.676300       -79.294784   
4                       Upper Beaches       43.680563       -79.292869   

         Venue Category  
0                 Trail  
1     Health Food Store  
2                   Pub  
3  Other Great Outdoors  
4          Neighborhood 

## Analysis of the venues

In [18]:
 print(tor_venues['Neighborhood'].head())
 np.unique(tor_venues['Neighborhood'].values)

0    The Beaches
1    The Beaches
2    The Beaches
3    The Beaches
4    The Beaches
Name: Neighborhood, dtype: object


array(['Adelaide, King, Richmond', 'Berczy Park',
       'Brockton, Exhibition Place, Parkdale Village',
       'Business Reply Mail Processing Centre 969 Eastern',
       'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara',
       'Cabbagetown, St. James Town', 'Central Bay Street',
       'Chinatown, Grange Park, Kensington Market', 'Christie',
       'Church and Wellesley', 'Commerce Court, Victoria Hotel',
       'Davisville', 'Davisville North',
       'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West',
       'Design Exchange, Toronto Dominion Centre',
       'Dovercourt Village, Dufferin',
       'First Canadian Place, Underground city',
       'Forest Hill North, Forest Hill West',
       'Harbord, University of Toronto',
       'Harbourfront East, Toronto Islands, Union Station',
       'Harbourfront, Regent Park', 'High Park, The Junction South',
       'Lawrence Park', 'Little Portugal, Trinity',
     

In [19]:
# one hot encoding
tor_venues_hot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
print(tor_venues_hot.shape)
# add neighborhood column back to dataframe
tor_venues_hot['Neighbourhood'] = tor_venues['Neighborhood']
print(tor_venues_hot.shape)
# move neighborhood column to the first column
fixed_columns = [tor_venues_hot.columns[-1]] + list(tor_venues_hot.columns[:-1])
tor_venues_hot = tor_venues_hot[fixed_columns]

print(tor_venues_hot.shape)
tor_venues_hot.head()

(1705, 231)
(1705, 232)
(1705, 232)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
tor_venues_group = tor_venues_hot.groupby('Neighbourhood').mean().reset_index()
print(tor_venues_group.shape)
tor_venues_group.head()

(38, 232)


,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.03,0.0,...,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.01,0.0,0.0
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.00,0.0,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.00,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.00,0.0,0.0


## Finding the top venues for each  Neighbourhood

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [22]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = tor_venues_group['Neighbourhood']

for ind in np.arange(tor_venues_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_venues_group.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Bar,Steakhouse,Thai Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Café,Steakhouse
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Café,Coffee Shop,Gym,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Pizza Place,Auto Workshop,Garden Center,Garden
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Plane,Bar


## Clustering Analysis

In [23]:
# set number of clusters
kclusters = 5

tor_venues_clustering = tor_venues_group.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 4, 0, 0])

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_tor_cut_merge = df_tor_cut.copy()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_tor_cut_merge = df_tor_cut_merge.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

df_tor_cut_merge.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Sandwich Place
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tor_cut_merge['Latitude'], df_tor_cut_merge['Longitude'], df_tor_cut_merge['Neighbourhood'], df_tor_cut_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [26]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 0, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store
42,East Toronto,0,Brewery,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Sandwich Place
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant
45,Central Toronto,0,Breakfast Spot,Hotel,Food & Drink Shop,Clothing Store,Dance Studio
46,Central Toronto,0,Coffee Shop,Clothing Store,Yoga Studio,Furniture / Home Store,Park
47,Central Toronto,0,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Pharmacy
49,Central Toronto,0,Coffee Shop,Pub,Fried Chicken Joint,Restaurant,Bagel Shop
51,Downtown Toronto,0,Coffee Shop,Park,Café,Restaurant,Italian Restaurant
52,Downtown Toronto,0,Coffee Shop,Japanese Restaurant,Gay Bar,Sushi Restaurant,Restaurant
53,Downtown Toronto,0,Coffee Shop,Café,Park,Bakery,Restaurant


In [27]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 1, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,Central Toronto,1,Playground,Park,Restaurant,Yoga Studio,Discount Store
50,Downtown Toronto,1,Park,Playground,Trail,Building,Yoga Studio


In [28]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 2, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
44,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Dog Run


In [29]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 3, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,Central Toronto,3,Music Venue,Garden,Yoga Studio,Fast Food Restaurant,Farmers Market


In [30]:
df_tor_cut_merge.loc[df_tor_cut_merge['Cluster Labels'] == 4, df_tor_cut_merge.columns[[1] + list(range(5, df_tor_cut_merge.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
37,East Toronto,4,Health Food Store,Other Great Outdoors,Neighborhood,Pub,Trail
